In [2]:
import cv2
import pandas as pd
import PIL.Image as Image
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from keras import models, Sequential, layers
from keras.models import load_model
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'cv2'

In [2]:
print(f"cv2 version is {cv2.__version__}")
print(f"numpy version is {np.__version__}")
print(f"tensorflow version is {tf.__version__}")
print(f"keras version is {keras.__version__}")
print(f"pandas version is {pd.__version__}")

cv2 version is 4.10.0
numpy version is 1.26.4
tensorflow version is 2.16.1
keras version is 3.3.3
pandas version is 2.2.2


In [3]:
def show_Image(name, image):
    cv2.imshow(name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [4]:
X , y = [], []
IMAGE_FOLDER = "../Captcha\ Processing/generated_single_images"
for name in os.listdir(IMAGE_FOLDER):
    path = os.path.join(IMAGE_FOLDER, name)
    image = cv2.imread(path)
    try:
        if image.shape == (50,50,3):
            X.append(image)
            y.append(name.split("_")[0])
    except:
        continue

In [5]:
X = np.array(X)
y = np.array(y)

In [6]:
unique = np.unique(y)

In [7]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_one_hot = keras.utils.to_categorical(y_encoded)

In [8]:
train_x,test_x , train_y,test_y = train_test_split(X,y_one_hot,test_size = 0.2)

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

(31704, 50, 50, 3)
(31704, 32)
(7927, 50, 50, 3)


In [9]:
train_x = train_x/255
test_x = test_x/255

In [10]:

def train_model():
    cnn = Sequential([
        layers.Conv2D(16 ,3,padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(32,3,padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64,3,padding="same", activation="relu"),
        layers.MaxPooling2D(),
        
        layers.Flatten(),
        layers.Dense(200, activation = "relu"),
        layers.Dense(100, activation = "relu"),
        layers.Dense(len(unique),activation="softmax")
    ])
    
    cnn.compile(optimizer = "adam",
               loss = keras.losses.CategoricalCrossentropy(),
               metrics = ["accuracy"])
    
    cnn.fit(train_x, train_y, epochs=20)
    return cnn

In [11]:
if "captcha_vision_model.keras" not in os.listdir("./"):
    cnn = train_model()
    cnn.save("captcha_vision_model.keras")
else:
    cnn = load_model("captcha_vision_model.keras")

/Users/user/Documents/Machine_Learning_Practice/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 14 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [12]:
cnn.evaluate(test_x,test_y)

248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9910 - loss: 0.0408


[0.03932895511388779, 0.9911693930625916]

In [13]:
y_pred = cnn.predict(test_x)


# cr = classification_report(y_true=test_y, y_pred=y_pred)

248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step


In [14]:
print(y_pred.shape)

(7927, 32)


In [17]:
pred_y = []
y_test = []
for i in range(len(y_pred)):
    pred_y.append(unique[np.argmax(y_pred[i])])
    y_test.append(unique[np.argmax(test_y[i])])

pred_y = np.array(pred_y)
y_test = np.array(y_test)


In [18]:
print(pred_y.shape)

(7927,)


In [19]:
cr = classification_report(y_pred=pred_y, y_true=y_test)

In [20]:
print(cr)

              precision    recall  f1-score   support

           2       1.00      1.00      1.00       244
           3       1.00      1.00      1.00       250
           4       1.00      1.00      1.00       252
           5       0.97      1.00      0.98       253
           6       1.00      1.00      1.00       240
           7       1.00      1.00      1.00       277
           8       1.00      1.00      1.00       248
           9       1.00      1.00      1.00       268
           A       1.00      1.00      1.00       227
           B       1.00      1.00      1.00       237
           C       1.00      0.99      0.99       259
           D       1.00      1.00      1.00       229
           E       1.00      0.99      0.99       240
           F       1.00      1.00      1.00       235
           G       0.95      1.00      0.98       245
           H       1.00      1.00      1.00       245
           J       0.99      0.98      0.98       243
           K       0.99    